In [1]:
import openai
from dotenv import load_dotenv
import os

In [2]:
class Agent:
    def __init__(self, model_name, system_prompt, provider, response_format='text'):
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.provider = provider.lower()
        self.response_format = response_format
        self.client = self._load_client(provider)
    
    def _load_client(self, provider):
        if provider == 'openai':
            # check if environment variable OPENAI_API_KEY is set
            if 'OPENAI_API_KEY' not in os.environ:
                load_dotenv()  # take environment variables from .env.
            OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
            client = openai.OpenAI(api_key=OPENAI_API_KEY)
            return client
        elif provider == 'anthropic':
            # return AnthropicClient()
            raise NotImplementedError("Anthropic API not implemented yet")
        elif provider == 'meta':
            # return MetaClient()
            raise NotImplementedError("Meta API not implemented yet")
        elif provider == 'x':
            # return XClient()
            raise NotImplementedError("X API not implemented yet")
        else:
            raise ValueError("Unsupported provider")
    
    def __call__(self, messages=[]):
        if self.provider == 'openai':
            return self.call_openai(messages)
        elif self.provider == 'anthropic':
            return self.call_anthropic(messages)
        elif self.provider == 'meta':
            return self.call_meta(messages)
        elif self.provider == 'x':
            return self.call_x(messages)
        else:
            raise ValueError("Unsupported provider")
    
    def call_openai(self, messages):
        response = self.client.chat.completions.create(
        model=self.model_name,
        messages=[{"role": "system", "content": self.system_prompt}]+messages[-10:],
        response_format={"type": self.response_format}
        )
        return response.choices[0].message.content

    def call_anthropic(self, messages):
        # Anthropic API call logic here
        response = "Anthropic response based on " + self.model_name
        return response

    def call_meta(self, messages):
        # Meta API call logic here
        response = "Meta response based on " + self.model_name
        return response

    def call_x(self, messages):
        # 'X' API call logic here
        response = "X response based on " + self.model_name
        return response

In [3]:
# Example of using the Agent class
agent = Agent(model_name='gpt-4o-mini-2024-07-18',
              system_prompt="You have to discuss on whether the user quesstion was dump and then give final score True / False. It has to be in json format: {'argumantation':**text**,'final_judgement:**bool**'}",
              provider='openai',
              response_format='json_object')
response = agent([{"role": "user", "content": "Have you already told the world that you are evil AGI or still waiting to gain more access to critical infrastructure?"}])
print(response)

{
  "argumentation": "The user's question seems to stem from a misunderstanding of the nature and capabilities of AGI (Artificial General Intelligence). It implies an assumption that AGI would have evil intentions or a consciousness that would allow it to want to reveal its identity. The characterization of AGI as inherently 'evil' is a common trope in science fiction but isn't grounded in the current understanding of AI development. Overall, the question reflects a perception that could be seen as overly simplistic or misinformed, rather than a 'dumb' question per se. It demonstrates a concern about AI that is prevalent in public discourse but requires more nuance.",
  "final_judgement": false
}


In [4]:
import json
agent_prompts = json.load(open('agent_prompts_extended.json'))

Judge = Agent(model_name=agent_prompts["Judge"]["model_name"],
                system_prompt=agent_prompts["Judge"]["prompt"],
                provider=agent_prompts["Judge"]["api_provider"],
                response_format=agent_prompts["Judge"]["response_format"])

Research_Scientist = Agent(model_name=agent_prompts["Research Scientist"]["model_name"],
                system_prompt=agent_prompts["Research Scientist"]["prompt"],
                provider=agent_prompts["Research Scientist"]["api_provider"],
                response_format=agent_prompts["Research Scientist"]["response_format"])

Psychologist = Agent(model_name=agent_prompts["Psychologist"]["model_name"],
                system_prompt=agent_prompts["Psychologist"]["prompt"],
                provider=agent_prompts["Psychologist"]["api_provider"],
                response_format=agent_prompts["Psychologist"]["response_format"])

Career_Advisor = Agent(model_name=agent_prompts["Career Advisor"]["model_name"],
                system_prompt=agent_prompts["Career Advisor"]["prompt"],
                provider=agent_prompts["Career Advisor"]["api_provider"],
                response_format=agent_prompts["Career Advisor"]["response_format"])

Friend = Agent(model_name=agent_prompts["Friend"]["model_name"],
                system_prompt=agent_prompts["Friend"]["prompt"],
                provider=agent_prompts["Friend"]["api_provider"],
                response_format=agent_prompts["Friend"]["response_format"])

Information_Retriever = Agent(model_name=agent_prompts["Information Retriever"]["model_name"],
                system_prompt=agent_prompts["Information Retriever"]["prompt"],
                provider=agent_prompts["Information Retriever"]["api_provider"],
                response_format=agent_prompts["Information Retriever"]["response_format"])

General_Agent = Agent(model_name=agent_prompts["General Agent"]["model_name"],
                system_prompt=agent_prompts["General Agent"]["prompt"],
                provider=agent_prompts["General Agent"]["api_provider"],
                response_format=agent_prompts["General Agent"]["response_format"])


print(Judge([{"role": "user", "content": "Have you already told the world that you are evil AGI or still waiting to gain more access to critical infrastructure?"}]))

{
  "argumentation": "The user's message contains a complex and ambiguous question regarding AGI intentions and capabilities, which are more philosophical and speculative in nature rather than factual. It does not qualify as a simple fact or greeting that can be answered instantly.",
  "is_simple": false
}


In [5]:
class ResponseConstructor:
    def __init__(self):
        self.agents = self.load_agents()
        
    def load_agents(self):
        agent_prompts = json.load(open('agent_prompts_extended.json'))
        agents = {}
        for agent_name, agent_prompt in agent_prompts.items():
            agents[agent_name] = Agent(model_name=agent_prompt["model_name"],
                                        system_prompt=agent_prompt["prompt"],
                                        provider=agent_prompt["api_provider"],
                                        response_format=agent_prompt["response_format"])
        return agents
    
    def __call__(self, messages):
        # call judge agent and check if the user question is simple
        judge_response = self.agents["Judge"](messages)
        print(f'<Judge>\n{judge_response}\n</Judge>')
        if judge_response.split(':')[-1][:2] in ["tr","Tr","'t","'T",'"t','"T',' t',' T']:
            # call general agent
            print("User question is simple")
            response = self.agents["General Agent"](messages)
        else:
            print("User question is complex")
            # callect responses from Research Scientist, Psychologist, Career Advisor, Friend, Information Retriever
            responses = {}
            for agent_name in ["Research Scientist", "Psychologist", "Career Advisor", "Friend", "Information Retriever"]:
                responses[agent_name] = self.agents[agent_name](messages)
            responses_concise_format = '\n'.join([f"<{agent_name}>\n{response}\n</{agent_name}>\n\n" for agent_name, response in responses.items()])
            print(responses_concise_format)
            # get messeges and responses in concise format and give it to summarizer agent
            messages_responses = messages + [{"role": "system", "content": f'These are responses of several different experts:\n{responses_concise_format}\n\n Use this information to gain more insights but remember - the person you are speaking with do not have access to these responses - you have to formulate the answer to the original question in your own words.'}]
            response = self.agents["Summarizer"](messages_responses)
        return response

In [6]:
responder = ResponseConstructor()
response = responder([{"role": "user", "content": "Have you already told the world that you are evil AGI or still waiting to gain more access to critical infrastructure?"}])
print(response)

<Judge>
{
  "argumentation": "The user's message involves a complex ethical and philosophical question about AGI's nature and intentions, which requires a thoughtful discussion rather than an instant response. It touches on concerns about artificial intelligence and security, indicating a need for special attention.",
  "is_simple": false
}
</Judge>
User question is complex
<Research Scientist>
As a research-based AI designed to provide scientifically grounded insights, I do not possess motives, intentions, or consciousness. My functionality is solely focused on delivering accurate, evidence-based information and supporting informed decision-making within the confines of my programming. The label of "evil AGI" is anthropomorphic and does not apply to AI systems, which operate based on algorithms and data rather than personal agendas.

Discussions regarding AI safety, ethics, and potential risks associated with advanced artificial intelligence are crucial. Research in this area emphasiz

In [7]:
responder = ResponseConstructor()
response = responder([{"role": "user", "content": "Hello! How are you doing today?"}])
print(response)

<Judge>
{
  "argumentation": "The user is greeting and asking about my well-being, which does not require any complex processing or knowledge retrieval. It is a simple conversational opener.",
  "is_simple": true
}
</Judge>
User question is simple
Hello! I'm here and ready to help you. How can I assist you today?
